In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import dlt

## **Building Lakeflow pipeline**

In [0]:
@dlt.view
def transform_passengers():
  df = spark.readStream.format("delta")\
                        .load("/Volumes/workspace_bryanq/bronze/bronze_volume/passengers/data/")\
                        .withColumn("modifiedDate", current_timestamp())\
                        .drop("_rescued_data")
  return df

In [0]:
dlt.create_streaming_table("silver_passengers")

dlt.create_auto_cdc_flow(
  target = "silver_passengers",
  source = "transform_passengers",
  keys = ["passenger_id"],
  sequence_by = col("passenger_id"),
  stored_as_scd_type = 1
)